In [1]:
%load_ext bigdata

In [2]:
%hive_start

In [3]:
%timeout 300

In [4]:
!hadoop fs -rm -f -r tmp;

In [5]:
## Borra la carpeta si existe
!hdfs dfs -rm -r -f /tmp/output
## Crea la carpeta drivers en el HDFS
!hdfs dfs -mkdir /tmp/output
## Copia los archivos al HDFS
!hdfs dfs -copyFromLocal data.tsv  /tmp/output/data.tsv
## Lista los archivos al HDFS para verificar
## que los archivos fueron copiados correctamente.
!hdfs dfs -ls /tmp/output/*

-rw-r--r--   1 root supergroup        637 2020-01-20 02:27 /tmp/output/data.tsv


In [6]:
%%hive
DROP TABLE IF EXISTS data;

CREATE TABLE data (letra       STRING,
                           fecha        STRING,
                           numero      INT)

ROW FORMAT DELIMITED
FIELDS TERMINATED BY '\t';
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
LOAD DATA LOCAL INPATH 'data.tsv' INTO TABLE data;

DROP TABLE IF EXISTS data;
OK
Time taken: 20.398 seconds
CREATE TABLE data (letra       STRING,
                           fecha        STRING,
                           numero      INT)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY '\t';
OK
Time taken: 1.567 seconds
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
NoViableAltException(58@[])
	at org.apache.hadoop.hive.ql.parse.HiveParser.statement(HiveParser.java:1300)
	at org.apache.hadoop.hive.ql.parse.ParseDriver.parse(ParseDriver.java:208)
	at org.apache.hadoop.hive.ql.parse.ParseUtils.parse(ParseUtils.java:77)
	at org.apache.hadoop.hive.ql.parse.ParseUtils.parse(ParseUtils.java:70)
	at org.apache.hadoop.hive.ql.Driver.compile(Driver.java:468)
	at org.apache.hadoop.hive.ql.Driver.compileInternal(Driver.java:1317)
	at org.apache.hadoop.hive.ql.Driver.runInternal(Driver.java:1457)
	at org.apache.hadoop.hive.ql.Driver.run(Driver.java:1237)
	at org.apache.hadoop.hive.ql.Driver.run(Driver.java:1227)


In [7]:
%%hive
--
-- para guardar la consulta se debe almacenar en un tabla
DROP TABLE IF EXISTS salida;
CREATE TABLE salida
AS
SELECT letra, fecha, numero
    FROM data
ORDER BY
    letra,
    numero;

--
-- para guardar la consulta se debe almacenar en un tabla
DROP TABLE IF EXISTS salida;
OK
Time taken: 0.029 seconds
CREATE TABLE salida
AS
SELECT letra, fecha, numero
    FROM data
ORDER BY
    letra,
    numero;
Query ID = root_20200120022807_d71294bd-01ff-414b-acb7-ee188533c816
Total jobs = 1
Launching Job 1 out of 1
Number of reduce tasks determined at compile time: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1579487079242_0001, Tracking URL = http://54950a17e4bf:8088/proxy/application_1579487079242_0001/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1579487079242_0001
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 1
2020-01-20 02:28:32,799 Stage-1 map = 0%,  reduce = 0%
2020-01

In [8]:
%%hive
DROP TABLE IF EXISTS data;

DROP TABLE IF EXISTS data;
OK
Time taken: 0.885 seconds


In [9]:
%%hive
-- 
INSERT OVERWRITE DIRECTORY 'tmp/000000_0'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
SELECT
    *
FROM
    salida;

-- 
INSERT OVERWRITE DIRECTORY 'tmp/000000_0'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
SELECT
    *
FROM
    salida;
Query ID = root_20200120022909_8fe84801-d31c-4cd5-8870-49470983e121
Total jobs = 3
Launching Job 1 out of 3
Number of reduce tasks is set to 0 since there's no reduce operator
Starting Job = job_1579487079242_0002, Tracking URL = http://54950a17e4bf:8088/proxy/application_1579487079242_0002/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1579487079242_0002
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 0
2020-01-20 02:29:22,682 Stage-1 map = 0%,  reduce = 0%
2020-01-20 02:29:30,915 Stage-1 map = 100%,  reduce = 0%, Cumulative CPU 2.7 sec
MapReduce Total cumulative CPU time: 2 seconds 700 msec
Ended Job = job_1579487079242_0002
Stage-3 is selected by condition resolver.
Stage-2 is filtered out by condition resolver.
Stage-4 is filtered out by condition resolver.
Moving data to directory hdfs://0.0.0.0:9000/user/root/tmp/0000

In [10]:
!hadoop fs -copyToLocal ./tmp/000000_0 output

In [11]:
%hive_quit